## Despliegue de trabajos ETL de lago de datos mediante CDK Pipelines

Muchas organizaciones están creando lagos de datos en AWS, que proporciona la cartera de servicios más segura, escalable, completa y rentable. Como cualquier proyecto de desarrollo de aplicaciones, un lago de datos debe responder a una pregunta fundamental: "¿Cuál es la estrategia DevOps?". Definir una estrategia DevOps para un lago de datos requiere una amplia planificación y múltiples equipos. Esto suele requerir múltiples ciclos de desarrollo y pruebas antes de madurar lo suficiente como para soportar un lago de datos en un entorno de producción. Si una organización no cuenta con las personas, los recursos y los procesos adecuados, esto puede volverse rápidamente desalentador.

¿Y si su equipo de ingeniería de datos utiliza bloques de construcción básicos para encapsular la infraestructura del lago de datos y los trabajos de procesamiento de datos? Aquí es donde CDK Pipelines aporta todas las ventajas de la infraestructura como código (IaC). CDK Pipelines es una biblioteca de construcción de alto nivel dentro del kit de desarrollo en la nube de AWS (AWS CDK) que facilita la configuración de una canalización de implementación continua para sus aplicaciones de AWS CDK. AWS CDK proporciona automatización esencial para sus canalizaciones de lanzamiento, de manera que su equipo de desarrollo y operaciones siga siendo ágil y se centre en desarrollar y entregar aplicaciones en el lago de datos.

#### Lagos de datos en AWS

Un lago de datos es un repositorio centralizado donde puede almacenar todos sus datos estructurados y no estructurados a cualquier escala. Almacene sus datos tal cual, sin tener que estructurarlos primero, y ejecute diferentes tipos de análisis, desde paneles y visualizaciones hasta procesamiento de big data, análisis en tiempo real y aprendizaje automático, con el fin de orientar mejores decisiones. Para saber más sobre los lagos de datos, consulte ¿Qué es un lago de datos?

Diseñamos un lago de datos con los siguientes elementos:

* Almacenamiento seguro de datos
* Catalogación de datos en un repositorio central
* Movimiento de datos
* Análisis de datos

La siguiente figura representa nuestro lago de datos:

<img src="https://d2908q01vomqb2.cloudfront.net/7719a1c782a1ba91c031a682a0a2f8658209adbf/2021/06/18/Aws-cdk-pipelines-blog-datalake-data_lake.png" alt="image" width="900">


Utilizamos tres buckets de Amazon Simple Storage Service (Amazon S3):

* **raw** - Almacena los datos de entrada en su formato original
* **conformed** - Almacena los datos que cumplen los requisitos de calidad del lago de datos
* **purpose-built** - Almacena los datos que están listos para ser consumidos por aplicaciones o consumidores del lago de datos.

El lago de datos cuenta con un productor en el que ingerimos datos en el raw bucket a intervalos periódicos. Utilizamos las siguientes herramientas: AWS Glue procesa y analiza los datos. AWS Glue Data Catalog persiste los metadatos en un repositorio central. AWS Lambda y AWS Step Functions programan y orquestan los trabajos de extracción, transformación y carga (ETL) de AWS Glue. Amazon Athena se utiliza para consultas y análisis interactivos. Por último, utilizamos varios servicios de AWS para el registro, la monitorización, la seguridad, la autenticación, la autorización, las alertas y las notificaciones.

Una práctica habitual en los lagos de datos es disponer de varios entornos, como los de desarrollo, prueba y producción. Aplicar el principio de IaC a los lagos de datos aporta la ventaja de ejecuciones coherentes y repetibles en varios entornos, una infraestructura autodocumentada y una mayor flexibilidad en la administración de recursos. AWS CDK ofrece construcciones de alto nivel para su uso con todos nuestros recursos de lago de datos. Esto simplifica el uso y agiliza la implementación.

Antes de explorar la implementación, conozcamos mejor cómo utilizamos nuestro lago de datos.

### La solución	

Nuestro objetivo es implementar una solución CI/CD que automatice el aprovisionamiento de los recursos de infraestructura del lago de datos y despliegue los trabajos ETL de forma interactiva. Lo conseguimos de la siguiente manera:

1. Aplicando el principio de diseño de separación de preocupaciones (SoC) a la infraestructura del lago de datos y a los trabajos ETL a través de repositorios de código fuente dedicados.

2. Un modelo de implementación centralizado que utiliza canalizaciones CDK

3. Canalizaciones ETL habilitadas para AWS CDK desde el principio.

Infraestructura de lago de datos
El aprovisionamiento de nuestra infraestructura de lago de datos incluye buckets de Amazon S3, políticas de buckets de S3, claves de cifrado de AWS Key Management Service (KMS), Amazon Virtual Private Cloud (Amazon VPC), subredes, tablas de rutas, grupos de seguridad, endpoints de VPC y secretos en AWS Secrets Manager. El siguiente diagrama lo ilustra.

<br>
<img src="https://d2908q01vomqb2.cloudfront.net/7719a1c782a1ba91c031a682a0a2f8658209adbf/2021/07/02/Aws-cdk-pipelines-blog-datalake-infra-3.png" alt="image" width="900">

### Trabajos de ETL de lago de datos

Para nuestros trabajos ETL, procesamos datos de registro de viajes de TLC de Nueva York. La siguiente figura muestra nuestro proceso ETL, en el que ejecutamos dos trabajos ETL dentro de una máquina de estados Step Functions.

<img src="https://d2908q01vomqb2.cloudfront.net/7719a1c782a1ba91c031a682a0a2f8658209adbf/2021/07/09/Aws-cdk-pipelines-blog-datalake-Copy-of-etl.png" alt="image" width="900">

He aquí algunos detalles importantes:

1. Un servidor de archivos sube archivos al raw bucket de S3 del lago de datos. El servidor de archivos es un productor de datos y una fuente para el lago de datos. Suponemos que los datos se envían al bucket sin procesar.

2. Amazon S3 activa una notificación de evento a la función Lambda.

3. La función inserta un elemento en la tabla de Amazon DynamoDB para realizar un seguimiento del estado de procesamiento del archivo. El primer estado escrito indica el inicio de la función de pasos de AWS.

4. La función inicia la máquina de estados.

5. La máquina de estados ejecuta un trabajo de AWS Glue (Apache Spark).

6. El trabajo procesa los datos de entrada de la zona sin procesar a la zona conformada del lago de datos. El trabajo también convierte los datos de entrada CSV en datos con formato Parquet.

7. El trabajo actualiza la tabla Data Catalog con los metadatos del archivo Parquet conformado.

8. Un segundo trabajo de AWS Glue (Apache Spark) procesa los datos de entrada de la zona conformada a la zona creada específicamente para el lago de datos.

9. El trabajo obtiene las reglas de transformación ETL del bucket de código de Amazon S3 y transforma los datos de entrada.

10. El trabajo almacena el resultado en formato Parquet en la zona específica.

11. El trabajo actualiza la tabla Data Catalog con los metadatos del archivo Parquet creado específicamente.

12. El trabajo actualiza la tabla DynamoDB y actualiza el estado del trabajo a completado.

13. Se envía una notificación de Amazon Simple Notification Service (Amazon SNS) a los suscriptores que indica que el trabajo se ha completado.

14. Los ingenieros o analistas de datos ya pueden analizar los datos a través de Athena.

Trataremos los formatos de datos, los trabajos de Glue, las lógicas de transformación ETL, la catalogación de datos, la auditoría, la notificación, la orquestación y el análisis de datos con más detalle en el repositorio de GitHub de AWS CDK Pipelines for Data Lake ETL Deployment. Esto se tratará en la sección siguiente.

### Implementación centralizada

Ahora que tenemos la infraestructura del lago de datos y los trabajos ETL listos, vamos a definir nuestro modelo de implementación. Este modelo se basa en los siguientes principios de diseño:

* Una cuenta de AWS dedicada para ejecutar pipelines CDK.
* Una o más cuentas de AWS en las que se implementa el lago de datos.
* La infraestructura del lago de datos tiene un repositorio de código fuente dedicado. Normalmente, la infraestructura del lago de datos se implementa una sola vez y rara vez evoluciona. Por lo tanto, un repositorio de código dedicado proporciona una zona de aterrizaje para su lago de datos.
* Cada trabajo ETL tiene un repositorio de código fuente dedicado. Cada trabajo ETL puede tener requisitos únicos de servicio, orquestación y configuración de AWS. Por lo tanto, un repositorio de código fuente dedicado le ayudará a crear, implementar y mantener trabajos ETL de forma más flexible.

Organizamos nuestro repositorio de código fuente en tres ramas: dev (principal), test y prod. En la cuenta de despliegue, gestionamos tres pipelines CDK independientes y cada pipeline procede de una rama dedicada. Aquí elegimos un método de desarrollo de software basado en ramas para demostrar la estrategia en escenarios más complejos en los que las pruebas de integración y las capas de validación requieren intervención humana. Además, debido a su naturaleza manual, es posible que no vayan seguidas inmediatamente de su correspondiente lanzamiento o despliegue. Esto facilita la propagación de los cambios a través de los entornos sin bloquear las prioridades de desarrollo independientes. Lo conseguimos aislando los recursos entre entornos en la cuenta de despliegue central, lo que permite la gestión independiente de cada entorno y evita la contaminación cruzada durante las actualizaciones automutables de cada canalización. El siguiente diagrama ilustra este método.

<br>
<img src="https://d2908q01vomqb2.cloudfront.net/7719a1c782a1ba91c031a682a0a2f8658209adbf/2021/06/19/Aws-cdk-pipelines-blog-datalake-branch_strategy_etl.png" alt="image" width="820">

**Nota**: Esta estrategia de despliegue centralizado puede adoptarse para el desarrollo de software basado en troncales con una modificación mínima de la solución.

### Despliegue de trabajos ETL de lago de datos

La siguiente figura ilustra cómo utilizamos CDK Pipelines para implementar la infraestructura del lago de datos y los trabajos ETL desde una cuenta de implementación central. Este modelo sigue la nomenclatura estándar del CDK de AWS. Cada repositorio representa una definición de código de infraestructura de nube. Esto incluye la definición del constructo pipelines. Las canalizaciones tienen una o más acciones, como clonar el código fuente (acción de origen) y sintetizar la pila en una plantilla de AWS CloudFormation (acción de sintetización). Cada canalización tiene una o más etapas, como la prueba y la implementación. En el contexto de una aplicación AWS CDK, la construcción de canalizaciones es una pila como cualquier otra pila. Por lo tanto, cuando se implementa la aplicación AWS CDK, se crea una nueva canalización en AWS CodePipeline.

Esto proporciona una flexibilidad increíble con respecto a DevOps. En otras palabras, como desarrollador que conoce las API de AWS CDK, puede aprovechar la potencia y escalabilidad de los servicios de AWS como CodePipeline, AWS CodeBuild y AWS CloudFormation.

<br>

![image](https://d2908q01vomqb2.cloudfront.net/7719a1c782a1ba91c031a682a0a2f8658209adbf/2021/07/02/Aws-cdk-pipelines-blog-datalake-continuous_delivery_data_lake_etl-1.png)

He aquí algunos detalles importantes:

1. El administrador de DevOps registra el código en el repositorio.

2. El administrador de DevOps (con acceso elevado) facilita un despliegue manual único en un entorno de destino. El acceso elevado incluye privilegios administrativos en la cuenta de despliegue central y en los entornos AWS de destino.

3. CodePipeline escucha periódicamente los eventos de confirmación en los repositorios de código fuente. Esta es la naturaleza auto-mutante de CodePipeline. Está configurado para trabajar y puede actualizarse a sí mismo de acuerdo con la definición proporcionada.

4. Los cambios de código realizados en la rama principal del repositorio se despliegan automáticamente en el entorno de desarrollo del lago de datos.

5. Los cambios de código realizados en la rama de prueba del repositorio se despliegan automáticamente en el entorno de prueba.

6. Los cambios de código realizados en la rama repo prod se despliegan automáticamente en el entorno prod.

#### [Canalizaciones CDK para la implantación de infraestructuras de lagos de datos](https://github.com/aws-samples/aws-cdk-pipelines-datalake-infrastructure)
#### [CDK Pipelines for Data Lake ETL Deployment](https://github.com/aws-samples/aws-cdk-pipelines-datalake-infrastructure)